## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [1]:
import datetime

import pandas as pd
from dotenv import load_dotenv

load_dotenv()
from web3 import Web3

from notebooks import get_block_by_ts
from notebooks import get_twap_bpt_price
from config_file import CORE_POOLS
from config_file import Chains
from config_file import WEB3_INSTANCES

TARGET_BLOCKS = {}
timestamp_now = 1695952610
timestamp_2_weeks_ago = timestamp_now - (2 * 7 * 24 * 60 * 60)
datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
datetime_2_weeks_ago = datetime.datetime.fromtimestamp(timestamp_2_weeks_ago)

target_blocks = {}
bpt_twap_prices = {chain.value: {} for chain in Chains}
# Collect BPT prices for 2 weeks ago and now for all relevant pools and chains
for chain in Chains:
    target_blocks[chain.value] = (
        get_block_by_ts(timestamp_now, chain.value),  # Block now
        get_block_by_ts(timestamp_2_weeks_ago, chain.value)  # Block 2 weeks ago
    )
    print(f"Collecting bpt prices for {chain.value}")
    pools = CORE_POOLS.get(chain.value, None)
    if pools is None:
        continue
    for core_pool in pools:
        bpt_twap_prices[chain.value][core_pool] = get_twap_bpt_price(
            core_pool, chain.value, getattr(WEB3_INSTANCES, chain.value),
            start_date=datetime.datetime.fromtimestamp(timestamp_now), block_number=target_blocks[chain.value][0]
        )

## Fetching fees and other data from the Balancer subgraphs

In [2]:
from notebooks.fees_and_bribs.config_file import BALANCER_GRAPH_URLS
from notebooks import get_balancer_pool_snapshots
from typing import Dict

pool_snapshots = {}
for chain in Chains:
    pool_snapshots[chain.value] = (
        get_balancer_pool_snapshots(target_blocks[chain.value][0], BALANCER_GRAPH_URLS[chain.value]),  # now
        get_balancer_pool_snapshots(target_blocks[chain.value][1], BALANCER_GRAPH_URLS[chain.value]),  # 2 weeks ago
    )

## Extract fee data for CORE pools:


In [3]:
from collections import defaultdict
from decimal import Decimal
from notebooks import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: Chains, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.datetime) -> tuple[dict, dict]:
    # Iterate through snapshots now and 2 weeks ago and extract fee data, by subtracting today's fee data from 2 weeks ago
    # and then summing across all pools
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # Take first element of list, which is the most recent snapshot
        if not current_fees_snapshots or not fees_2_weeks_ago:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        bpt_price_usd = bpt_twap_prices[chain.value][pool]
        if bpt_price_usd is None:
            bpt_price_usd = 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT'] is not None and pool_snapshot_2_weeks_ago['pool'][
            'totalProtocolFeePaidInBPT'] is not None:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'])
            token_fees[pool_snapshot_now['pool']['id']].append({
                'symbol': pool_snapshot_now['pool']['symbol'],
                'token': pool_snapshot_now['pool']['symbol'],
                'token_fee': bpt_token_fee,
                'token_price': bpt_price_usd,
                'token_fee_in_usd': Decimal(bpt_token_fee) * bpt_price_usd,
                'token_addr': pool_snapshot_now['pool']['address'],
                'time_from': datetime_2_weeks_ago,
                'time_to': datetime_now,
                'chain': chain.value,
            })
        # Now collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
        else:
            bpt_price_usd = 0
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None))
                # Get twap token price from CoinGecko
                token_price = fetch_token_price_balgql(token_data['address'], chain.value, start_date) or 0
                token_fees[pool_snapshot_now['pool']['id']].append({
                    'symbol': pool_snapshot_now['pool']['symbol'],
                    'token': token_data['symbol'],
                    'token_fee': token_fee,
                    'token_price': token_price,
                    'token_fee_in_usd': Decimal(token_fee) * token_price if token_price is not None else 0,
                    'token_addr': token_data['address'],
                    'time_from': datetime_2_weeks_ago,
                    'time_to': datetime_now,
                    'chain': chain.value,
                })
        # Calculate non-BPT fees in USD
        fees[pool_snapshot_now['pool']['id']] = {
            'symbol': pool_snapshot_now['pool']['symbol'],
            'pool_addr': pool_snapshot_now['pool']['address'],
            'bpt_token_fee': round(bpt_token_fee, 2),
            # Get fee in USD by multiplying bpt_token_fee by price of BPT token taken from twap_bpt_price
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(sum([x['token_fee_in_usd'] for x in
                                            token_fees[
                                                pool_snapshot_now['pool']['symbol']]]) if bpt_price_usd == 0 else 0, 2),
            'time_from': datetime_2_weeks_ago,
            'time_to': datetime_now,
            'chain': chain.value,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']]
        }
    return fees


collected_fees = {}

for chain in Chains:
    core_pools = CORE_POOLS.get(chain.value, None)
    if not core_pools:
        continue
    collected_fees[chain.value] = collect_fee_info(core_pools, chain, pool_snapshots[chain.value][0],
                                                   pool_snapshots[chain.value][1], datetime_now)
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**collected_fees[Chains.MAINNET.value], **collected_fees[Chains.ARBITRUM.value],
              **collected_fees[Chains.POLYGON.value], **collected_fees[Chains.BASE.value],
              **collected_fees[Chains.AVALANCHE.value]}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

In [4]:
# Remove `token_fees` field from dataframe, as it's too big
joint_fees_df_copy = joint_fees_df.drop(columns=['token_fees'])
# Display all rows in dataframe
pd.set_option('display.max_rows', 1000)
joint_fees_df_copy.sort_values(by=['chain'], ascending=False)

,symbol,pool_addr,bpt_token_fee,bpt_token_fee_in_usd,token_fees_in_usd,time_from,time_to,chain
0xee278d943584dd8640eaf4cc6c7a5c80c0073e85000200000000000000000bc7,ECLP-WMATIC-MATICX,0xee278d943584dd8640eaf4cc6c7a5c80c0073e85,1254.62,659.76,0,2023-09-15 04:56:50,2023-09-29 04:56:50,polygon
0xf0ad209e2e969eaaa8c882aac71f02d8a047d5c2000200000000000000000b49,ECLP-WMATIC-stMATIC,0xf0ad209e2e969eaaa8c882aac71f02d8a047d5c2,3410.29,1800.78,0,2023-09-15 04:56:50,2023-09-29 04:56:50,polygon
0x93d199263632a4ef4bb438f1feb99e57b4b5f0bd0000000000000000000005c2,wstETH-WETH-BPT,0x93d199263632a4ef4bb438f1feb99e57b4b5f0bd,5.24,8467.25,0,2023-09-15 04:56:50,2023-09-29 04:56:50,mainnet
0x1ee442b5326009bb18f2f472d3e0061513d1a0ff000200000000000000000464,50rETH-50BADGER,0x1ee442b5326009bb18f2f472d3e0061513d1a0ff,116.14,6939.33,0,2023-09-15 04:56:50,2023-09-29 04:56:50,mainnet
0x1e19cf2d73a72ef1332c882f20534b6519be0276000200000000000000000112,B-rETH-STABLE,0x1e19cf2d73a72ef1332c882f20534b6519be0276,0.00,0.00,0,2023-09-15 04:56:50,2023-09-29 04:56:50,mainnet
0x36be1e97ea98ab43b4debf92742517266f5731a3000200000000000000000466,50wstETH-50ACX,0x36be1e97ea98ab43b4debf92742517266f5731a3,94.89,933.62,0,2023-09-15 04:56:50,2023-09-29 04:56:50,mainnet
0xf01b0684c98cd7ada480bfdf6e43876422fa1fc10002000000000000000005de,ECLP-wstETH-wETH,0xf01b0684c98cd7ada480bfdf6e43876422fa1fc1,0.00,0.00,0,2023-09-15 04:56:50,2023-09-29 04:56:50,mainnet
0x9f9d900462492d4c21e9523ca95a7cd86142f298000200000000000000000462,50rETH-50RPL,0x9f9d900462492d4c21e9523ca95a7cd86142f298,5.95,1165.36,0,2023-09-15 04:56:50,2023-09-29 04:56:50,mainnet
0x3ff3a210e57cfe679d9ad1e9ba6453a716c56a2e0002000000000000000005d5,STG/USDC,0x3ff3a210e57cfe679d9ad1e9ba6453a716c56a2e,8751.94,5930.07,0,2023-09-15 04:56:50,2023-09-29 04:56:50,mainnet
0x5f1f4e50ba51d723f12385a8a9606afc3a0555f5000200000000000000000465,50wstETH-50LDO,0x5f1f4e50ba51d723f12385a8a9606afc3a0555f5,14.53,764.84,0,2023-09-15 04:56:50,2023-09-29 04:56:50,mainnet


## Now let's calculate bribes paid to the pools and save them to csv

In [14]:
from notebooks.fees_and_bribs.config_file import MIN_AURA_BRIB
from notebooks.fees_and_bribs.config_file import REROUTE_CONFIG
from notebooks import calculate_aura_vebal_share

aura_vebal_share = calculate_aura_vebal_share(WEB3_INSTANCES.mainnet, target_blocks[Chains.MAINNET.value][0])

# Bribes are split per chain and per pool, with each pool getting a share of the bribe proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the bribes. That 10% will be distributed between aura and vebal, proportional to their share of the bribe.

FEES_TO_DISTRIBUTE = {
    Chains.MAINNET.value: Decimal(112945.77),
    Chains.ARBITRUM.value: Decimal(39481.82),
    Chains.POLYGON.value: Decimal(6592.87),
    Chains.BASE.value: Decimal(10260.35),
    Chains.GNOSIS.value: Decimal(2503.21),
    Chains.AVALANCHE.value: Decimal(10297),
}

# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_bribes(fees: Dict, chain: str, fees_to_distribute: Decimal) -> Dict[str, Dict]:
    pool_bribs = {}
    # Calculate pool share in fees
    dao_share = Decimal(0.15)
    vebal_share = Decimal(0.35)
    fees_to_distr_wo_dao_vebal = fees_to_distribute - (fees_to_distribute * dao_share) - (
            fees_to_distribute * vebal_share)
    # Calculate totals
    bpt_fees = sum([data['bpt_token_fee_in_usd'] for pool, data in fees.items()])
    token_fees = sum([data['token_fees_in_usd'] for pool, data in fees.items()])
    total_fees = bpt_fees + token_fees
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        pool_share = pool_fees / Decimal(total_fees)
        # If aura bribes is less than 500 USDC, we pay all bribes to balancer
        aura_bribes = round(pool_share * fees_to_distr_wo_dao_vebal * aura_vebal_share, 2)
        if aura_bribes <= Decimal(MIN_AURA_BRIB):
            aura_bribes = Decimal(0)
            bal_bribes = round(pool_share * fees_to_distr_wo_dao_vebal, 2)
        else:
            bal_bribes = round(pool_share * fees_to_distr_wo_dao_vebal * (1 - aura_vebal_share), 2)
        fees_to_dao = round(pool_share * fees_to_distribute * dao_share, 2)
        fees_to_vebal = round(pool_share * fees_to_distribute * vebal_share, 2)
        # Split fees between aura and bal fees
        pool_bribs[pool] = {
            "chain": chain,
            "symbol": data['symbol'],
            "protocol_fees_collected": pool_fees,
            "fees_to_vebal": fees_to_vebal,
            "fees_to_dao": fees_to_dao,
            "total_bribes": aura_bribes + bal_bribes,
            "aura_bribes": aura_bribes,
            "bal_bribes": bal_bribes,
            "pool_addr": data['pool_addr'],
        }
    return pool_bribs

def re_route_bribs(bribes_data: Dict[str, Dict], chain: Chains) -> Dict[str, Dict]:
    """
    If pool is in re-route configuration, all bribes from that pool should be distributed to destination pool
      Ex: {source_pool: destination_pool}
    """
    if not chain.value in REROUTE_CONFIG:
        return bribes_data
    for pool_id, _data in bribes_data.items():
        if pool_id in REROUTE_CONFIG[chain.value]:
            # Re route everything to destination pool and set source pool bribes to 0
            bribes_data[REROUTE_CONFIG[pool_id]]['aura_bribes'] += _data['aura_bribes']
            bribes_data[REROUTE_CONFIG[pool_id]]['bal_bribes'] += _data['bal_bribes']
            bribes_data[pool_id]['aura_bribes'] = 0
            bribes_data[pool_id]['bal_bribes'] = 0
    return bribes_data


bribes = {}
for chain in Chains:
    if not collected_fees.get(chain.value):
        continue
    _bribs = calc_and_split_bribes(collected_fees[chain.value], chain.value,
                                                FEES_TO_DISTRIBUTE[chain.value])
    # Reroute bribes if needed
    bribs = re_route_bribs(_bribs, chain)
    bribes[chain.value] = bribs
    
joint_bribes_data = {**bribes[Chains.MAINNET.value], **bribes[Chains.ARBITRUM.value],
                     **bribes[Chains.POLYGON.value], **bribes[Chains.BASE.value], **bribes[Chains.AVALANCHE.value]}
# Sort by chain:
joint_bribes_data = {k: v for k, v in sorted(joint_bribes_data.items(), key=lambda item: item[1]['chain'])}
joint_bribes_df = pd.DataFrame.from_dict(joint_bribes_data, orient='index')
# Sort by chain
# Dump into csv and prefix with dates
joint_bribes_df.to_csv(f'./output/bribes_{datetime_2_weeks_ago.date()}_{datetime_now.date()}.csv')
# Don't show pool_addr
display = joint_bribes_df.drop(columns=['pool_addr'])
# Sort by chain and by protocol_fees collected
display.sort_values(by=['protocol_fees_collected'], ascending=False)
display

,chain,symbol,protocol_fees_collected,fees_to_vebal,fees_to_dao,total_bribes,aura_bribes,bal_bribes
0xade4a71bb62bec25154cfc7e6ff49a513b491e81000000000000000000000497,arbitrum,rETH-WETH-BPT,860.27,463.77,198.76,662.52,0,662.52
0x9791d590788598535278552eecd4b211bfc790cb000000000000000000000498,arbitrum,wstETH-WETH-BPT,2286.57,1232.68,528.29,1760.96,690.43,1070.53
0x4a2f6ae7f3e5d715689530873ec35593dc28951b000000000000000000000481,arbitrum,wstETH/rETH/cbETH,3251.97,1753.12,751.34,2504.46,981.94,1522.52
0x423a1323c871abc9d89eb06855bf5347048fc4a5000000000000000000000496,arbitrum,4POOL-BPT,447.85,241.43,103.47,344.90,0,344.90
0x32df62dc3aed2cd6224193052ce665dc181658410002000000000000000003bd,arbitrum,RDNT-WETH,18786.37,10127.62,4340.41,14468.03,5672.57,8795.46
0x0c8972437a38b389ec83d1e666b69b8a4fcf8bfd00000000000000000000049e,arbitrum,wstETH/rETH/sfrxETH,0.04,0.02,0.01,0.03,0,0.03
0xfd2620c9cfcec7d152467633b3b0ca338d3d78cc00000000000000000000001c,avalanche,sAVAX-WAVAX-BPT,2384.31,2337.13,1001.63,3338.76,1309.05,2029.71
0xc13546b97b9b1b15372368dc06529d7191081f5b00000000000000000000001d,avalanche,ggAVAX-WAVAX-BPT,566.36,555.15,237.92,793.08,0,793.08
0x9fa6ab3d78984a69e712730a2227f20bcc8b5ad900000000000000000000001f,avalanche,yyAVAX-WAVAX-BPT,545.26,534.47,229.06,763.53,0,763.53
0xb26f0e66317846bd5fe0cbaa1d269f0efeb05c9600000000000000000000001e,avalanche,USDC-USDT-BPT,180.77,177.19,75.94,253.13,0,253.13


## Compare data from the sheet with the data we collected
What does table below represent:
- aura_automated_bribe: bribes paid to aura per pool calculated in this notebook
- aura_actual_bribe: actual bribes paid to aura per pool from the sheet
- aura_bribe_delta: difference between aura_automated_bribe and aura_actual_bribe
- aura_bribe_delta_%: difference between aura_automated_bribe and aura_actual_bribe in %

Same thing for balancer side:
- bal_automated_bribe: bribes paid to balancer per pool calculated in this notebook
- bal_actual_bribe: actual bribes paid to balancer per pool from the sheet
- bal_bribe_delta: difference between bal_automated_bribe and bal_actual_bribe
- bal_bribe_delta_%: difference between bal_automated_bribe and bal_actual_bribe in %

In [6]:
from notebooks import fetch_all_pools_info

mapped_gauges = {}

for chain in Chains:
    pools_info = fetch_all_pools_info(chain.value)
    mapped_gauges[chain.value] = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for
                                  pool_info in pools_info}

In [7]:
# Load csv sheet to compare with the data collected above:
bribes_df = pd.read_csv('compare_sheet.csv')
# To dict:
bribes_to_compare = bribes_df.to_dict(orient='records')
# Each dict in list needs target to be Web3.toChecksumAddress
bribes_csv_checksummed = []
for bribe in bribes_to_compare:
    bribes_csv_checksummed.append({
        'target': Web3.to_checksum_address(bribe['target']),
        'platform': bribe['platform'],
        'amount': bribe['amount'],
    })
# These are bribes paid de-facto, so we need to convert them to the same format as we have in our data
bribes_final_csv = {}
for bribe in bribes_csv_checksummed:
    if bribe['target'] not in bribes_final_csv:
        bribes_final_csv[bribe['target']] = {}
    bribes_final_csv[bribe['target']][bribe['platform']] = Decimal(bribe['amount'])

In [8]:
# Now we need to compare the data we collected with the data from the sheet
# We need to compare bribes paid to each pool
joint_bribes_dict = joint_bribes_df.to_dict(orient='records')
bribes_delta = {}
for gauge, data in bribes_final_csv.items():
    for automated_bribe in joint_bribes_dict:
        filtered_gauges = mapped_gauges[automated_bribe['chain']]
        # If this condition is true, we can compare the bribes
        if filtered_gauges[automated_bribe['pool_addr']] == gauge:
            # Calculate delta between automated bribes and bribes from the sheet
            # Calculate delta between automated_bribe['aura_bribes'] and data['aura'] in %
            if automated_bribe['aura_bribes'] > data['aura']:
                aura_bribe_delta = automated_bribe['aura_bribes'] - data['aura']
                aura_bribe_delta_pct = (automated_bribe['aura_bribes'] - data['aura']) / data['aura'] * 100 if \
                    data['aura'] != 0 else 0
            else:
                aura_bribe_delta = data['aura'] - automated_bribe['aura_bribes']
                aura_bribe_delta_pct = (data['aura'] - automated_bribe['aura_bribes']) / data['aura'] * 100 if \
                    data['aura'] != 0 else 0
            if automated_bribe['bal_bribes'] > data['balancer']:
                bal_bribe_delta = automated_bribe['bal_bribes'] - data['balancer']
                bal_bribe_delta_pct = (automated_bribe['bal_bribes'] - data['balancer']) / data['balancer'] * 100 if \
                    data['balancer'] != 0 else 0
            else:
                bal_bribe_delta = data['balancer'] - automated_bribe['bal_bribes']
                bal_bribe_delta_pct = (data['balancer'] - automated_bribe['bal_bribes']) / data['balancer'] * 100 if \
                    data['balancer'] != 0 else 0
            bribes_delta[gauge] = {
                'symbol': automated_bribe['symbol'],
                'chain': automated_bribe['chain'],
                'aura_automated_bribe': automated_bribe['aura_bribes'],
                'aura_actual_bribe': round(data['aura'], 2),
                'aura_bribe_delta': round(aura_bribe_delta, 2),
                'aura_bribe_delta_%': f'{round(aura_bribe_delta_pct, 2)}%',
                'bal_automated_bribe': automated_bribe['bal_bribes'],
                'bal_actual_bribe': round(data['balancer'], 2),
                'bal_bribe_delta': round(bal_bribe_delta, 2),
                'bal_bribe_delta_%': f'{round(bal_bribe_delta_pct, 2)}%',
            }
# To df and print out without index:
bribes_delta_df = pd.DataFrame.from_dict(bribes_delta, orient='index')
bribes_delta_df.sort_values(by=['chain', 'aura_bribe_delta'], ascending=False)
# Also dump to csv
bribes_delta_df.to_csv(f'./output/bribes_delta_{datetime_2_weeks_ago.date()}_{datetime_now.date()}.csv')
bribes_delta_df

,symbol,chain,aura_automated_bribe,aura_actual_bribe,aura_bribe_delta,aura_bribe_delta_%,bal_automated_bribe,bal_actual_bribe,bal_bribe_delta,bal_bribe_delta_%
0x5C0F23A5c1be65Fa710d385814a7Fd1Bda480b1C,wstETH-WETH-BPT,mainnet,4010.64,1676.15,2334.49,139.28%,6218.60,3112.85,3105.75,99.77%
0x79eF6103A513951a3b25743DB509E267685726B7,B-rETH-STABLE,mainnet,0,2976.40,2976.40,100.00%,0.00,5527.60,5527.60,100.00%
0xee01c0d9c0439c94D314a6ecAE0490989750746C,swETH-WETH-BPT,mainnet,1709.86,0.00,1709.86,0%,2651.17,0.00,2651.17,0%
0x4D4264aebf65Bb1727bb5438E0b2Aaf86186DA50,ECLP-wstETH-cbETH,mainnet,0,0.00,0.00,0%,0.00,0.00,0.00,0%
0x183D73dA7adC5011EC3C46e33BB50271e59EC976,20WETH-80ALCX,mainnet,5664.75,3046.75,2618.00,85.93%,8783.34,5658.25,3125.09,55.23%
0x10a361766e64D7983a97202ac3a0F4cee06Eb717,B-staFiETH-WETH-Stable,mainnet,0,0.00,0.00,0%,0.00,0.00,0.00,0%
0xf720e9137baa9C7612e6CA59149a5057ab320cFa,GHO/USDT/USDC,mainnet,693.44,0.00,693.44,0%,1075.19,0.00,1075.19,0%
0xbf65b3fA6c208762eD74e82d4AEfCDDfd0323648,ankrETH/wstETH,mainnet,2610.70,2057.83,552.87,26.87%,4047.96,3821.68,226.28,5.92%
0x87012b0C3257423fD74a5986F81a0f1954C17a1d,50rETH-50BADGER,mainnet,3286.92,1279.78,2007.14,156.83%,5096.45,2376.73,2719.72,114.43%
0x2C2179abce3413E27BDA6917f60ae37F96D01826,50rETH-50RPL,mainnet,551.99,0.00,551.99,0%,855.88,0.00,855.88,0%
